<a href="https://colab.research.google.com/github/AbhinavRadhakrishnan/Gen-Ai-projects/blob/main/Pyndantic_ai_Agent_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building Ai Agent with Pydantic AI**

In [45]:
!pip install -q requests pydantic-ai "pydantic-ai-slim[groq]"

/usr/lib/python3.12/pathlib.py:404: RuntimeWarning: coroutine 'AbstractAgent.run' was never awaited
  parsed = [sys.intern(str(x)) for x in rel.split(sep) if x and x != '.']


In [46]:
import nest_asyncio
nest_asyncio.apply()
# python can do morethan 1 things at a time parallel

In [47]:
import requests


BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "your api"

def find_weather(city: str) -> dict :
  """ This function returns the current weather forecast for the given city"""
  units = "metric"
  params = {
      "q": city,
      "appid": API_KEY,
      "units": units
  }
  response = requests.get(BASE_URL, params=params)
  result = response.json()
  return result

In [48]:
output = find_weather("Kochi")
print(output)

{'coord': {'lon': 76.2602, 'lat': 9.9399}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 25.51, 'feels_like': 26.15, 'temp_min': 25.51, 'temp_max': 32.99, 'pressure': 1011, 'humidity': 78, 'sea_level': 1011, 'grnd_level': 1010}, 'visibility': 10000, 'wind': {'speed': 1.79, 'deg': 100}, 'clouds': {'all': 35}, 'dt': 1765181482, 'sys': {'type': 2, 'id': 267800, 'country': 'IN', 'sunrise': 1765155607, 'sunset': 1765197227}, 'timezone': 19800, 'id': 1273874, 'name': 'Kochi', 'cod': 200}


In [49]:
print(output["name"])
print(output["weather"][0]["description"].capitalize())
print(output["main"]["temp"])

Kochi
Scattered clouds
25.51


Pydantic AI AGENT


In [50]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [51]:
os.environ["GROQ_API_KEY"] = "your api"

In [52]:
#define output schema for tool
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [65]:
weather_agent = Agent(
    model = "groq:llama-3.1-8b-instant",
    model_settings = ModelSettings(temperature=0.2),
    output_type = str,
    system_prompt = (  "You are a helpful weather assistant. "
                        "Always use the 'get_weather_forecast' tool to answer user queries. "
                        "If the user enters ONLY a city name (e.g., 'Chennai', 'Paris', 'Tokyo'), "
                        "treat it as a request for the current weather in that city. "
                        "Never return placeholder or sample text. "
                        "Always call the tool with the given city name")

)

In [66]:
#weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx : RunContext, city: str) -> WeatherForecast:

    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "your api"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(url, params=params)
    result = response.json()

    return WeatherForecast(
        location=result["name"],
        description=result["weather"][0]["description"].capitalize(),
        temperature_celsius=result["main"]["temp"]
    )

In [67]:
agent_response = weather_agent.run_sync("What is the weather like in Chennai?")
print(agent_response.output)

The current weather in Chennai is Haze with a temperature of 28.57°C.


In [74]:
question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)


🌤️ Ask about the weather: chennai

📍 Forecast: The current weather in Chennai is Haze with a temperature of 28.57°C.


In [80]:
"""
Weather Assistant using Pydantic-AI and OpenWeatherMap API
This script creates a conversational agent that can respond to weather-related queries
using the OpenWeatherMap API and a Groq-hosted LLaMA model.
"""

# Allow nested async loops (useful for Jupyter or async environments)
import nest_asyncio
nest_asyncio.apply()

# Standard libraries
import os
import requests

# Pydantic model for structured data
from pydantic import BaseModel

# Core AI libraries from pydantic_ai
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

os.environ["GROQ_API_KEY"] = "your api"

# 1. Define the output schema of the tool using Pydantic
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

# 2. Create the AI agent using Groq’s LLaMA 3 model
weather_agent = Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt=(
        "You are a helpful weather assistant. Use the 'get_weather_forecast' tool to "
        "find current weather conditions for any city. Provide clean and friendly answers."
    )
)

# 3. Register a tool with the agent to fetch real-time weather using OpenWeatherMap API
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Tool: get_weather_forecast
    Description: Fetches current weather for a city using the OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Replace this with your own API key for production use
    api_key ="your api"

    # Query parameters
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    # Send request to weather API
    res = requests.get(url, params=params).json()

    # Return the formatted weather information
    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

# 4. Run continuous user interaction loop
if __name__ == "__main__":
    print("🌦️  Weather Assistant is ready! Type 'exit' to quit.")
    print("‒" * 50)
    while True:
        question = input("🌤️ Ask about the weather: ").strip()
        if question.lower() in {"exit", "quit", ""}:
            print("\n👋 Exiting weather assistant. Have a nice day!")
            break

        try:
            result = weather_agent.run_sync(question)
            print("\n📍 Forecast:", result.output)
        except Exception as e:
            print("⚠️ Error:", str(e))

        print("‒" * 50)

🌦️  Weather Assistant is ready! Type 'exit' to quit.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
🌤️ Ask about the weather: chennai

📍 Forecast: It's currently 28.57 degrees Celsius in Chennai with a haze condition.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
🌤️ Ask about the weather: Paris

📍 Forecast: It's currently overcast with a temperature of 14.14°C in Paris.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
🌤️ Ask about the weather: Kochi
⚠️ Error: status_code: 400, model_name: llama-3.1-8b-instant, body: {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=get_weather_forecast>{"city": "Kochi"}'}}
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
🌤️ Ask about the weather: london

📍 Forecast: It's currently 12.62°C in London with few clouds.
‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒‒
🌤️ Ask about t